In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import pandas as pd 
import numpy as np
from datetime import datetime 

import csv
import requests

from pairs_selection.api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap

In [75]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2020, 8, 9), datetime(2022, 8, 9)]
    )

In [76]:
tker_dict = val.get_tkers(settings)

In [92]:
valid_lst = val.get_valid_tkers(tker_dict)

there


In [95]:
print(valid_lst[0][0])

['symbol', 'name', 'exchange', 'assetType', 'ipoDate', 'delistingDate', 'status']


In [80]:
print(len(valid_lst))

12696


In [73]:
test_case = {"one": ["A", "B", "C"], "two": ["A", "B", "B2", "C"], "three": ["A", "C", "D"]}
test_lst = val.get_valid_tkers(test_case)
print(test_lst)

B was delisted in three
B2 was delisted in three
['A', 'C']


In [43]:
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=demo'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)


{'microsoft': ['MSFT', '0QYP.LON', 'MSF.DEX', 'MSF.FRK', 'MSFT34.SAO'], 'meta': ['META', 'METAX', 'META.LON', 'METEX.PAR', 'METALCO.BSE', 'MTCR', 'ALMTA.PAR', 'METALFORGE.BSE', 'AIU', '7ME.FRK']}


In [8]:
#test case on IBM
from pairs_selection.api_wrapper import alph_api_wrapper


function = "TIME_SERIES_INTRADAY_EXTENDED"
ticker_lst = ["IBM", "AAPL", "MSFT"]
interval = "60min"
slice = [f"year1month{i}" for i in range(1, 4)] #first three months of intraday data
outputsize = "compact"
test = alph_api_wrapper.intraday_url(settings, function, ticker_lst, interval, slice = slice, outputsize = outputsize)

test_url = test["IBM"][0]
test_obj = alph_api_wrapper.get_csv_data(test_url, pretty_print = 10)

['time', 'open', 'high', 'low', 'close', 'volume']
['2022-10-10 20:00:00', '117.65', '117.65', '117.65', '117.65', '905']
['2022-10-10 19:00:00', '117.7', '117.75', '117.7', '117.75', '1786']
['2022-10-10 18:00:00', '117.5', '117.5', '117.5', '117.5', '100']
['2022-10-10 17:00:00', '117.75', '118.4', '117.75', '117.75', '2159137']
['2022-10-10 16:00:00', '117.93', '118.25', '117.76', '117.8', '828077']
['2022-10-10 15:00:00', '118.17', '118.3309', '117.825', '117.97', '316454']
['2022-10-10 14:00:00', '117.2', '118.25', '117.04', '118.15', '527808']
['2022-10-10 13:00:00', '118.07', '118.2', '117.21', '117.22', '316137']
['2022-10-10 12:00:00', '118.28', '118.44', '117.62', '118.075', '487181']


In [ ]:
#get all tickers in the list 
full_dict = {}
for i in test.keys(): 
    for j in test[i].keys(): 
        
    #full_dict[i] = 

In [45]:
mth = len(test_obj)/3 #per month 
day = mth/30 #per day
t_hr = day/8 #per trading hour
print(mth, day, t_hr)

2761.3333333333335 92.04444444444445 11.505555555555556
